In [1]:
import sys
import torch
import cv2
import numpy as np
import pandas as pd
import os
from torchvision import transforms
from PIL import Image
import torch.nn as nn
from torchvision import models
from torchvision.models import EfficientNet_V2_L_Weights
from pynvml import nvmlInit, nvmlDeviceGetCount, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo, nvmlShutdown
from tqdm import tqdm
import pickle

In [2]:
def crop_faces(frames):
    """Crop faces from frames using Haar cascade. Return same-sized RGB frames."""
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    cropped = []

    for frame in frames:
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        if len(faces) > 0:
            x, y, w, h = max(faces, key=lambda f: f[2]*f[3])  # largest face
            face = frame[y:y+h, x:x+w]
            face = cv2.resize(face, (frame.shape[1], frame.shape[0]))  # Resize back to original size
            cropped.append(face)
        else:
            cropped.append(frame)  # fallback to original
    return cropped

In [3]:
def extract_frames_from_video(video_path, num_frames=60):
    """
    Extract a fixed number of frames evenly from a video.
    
    Args:
        video_path (str): Path to the video file.
        num_frames (int): Number of frames to extract.
        
    Returns:
        list: List of frames in RGB format (as numpy arrays).
    """
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Create indices evenly spaced through the video
    frame_idxs = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    frames = []
    
    for idx in frame_idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)  # Move to the desired frame
        ret, frame = cap.read()
        if not ret:
            break
        # Convert BGR (OpenCV default) to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    faces = crop_faces(frames)
    cap.release()
    return faces

In [4]:
preprocess = transforms.Compose([
    transforms.Resize(600),          # Resize so that the smaller side is 600 pixels
    transforms.CenterCrop(600),      # Then center crop to 600x600
    transforms.ToTensor(),           # Convert image to tensor with shape (3, H, W) in [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [5]:
# Load EfficientNetV2-L pretrained on ImageNet21k
model = models.efficientnet_v2_l(weights=EfficientNet_V2_L_Weights.IMAGENET1K_V1)

# Remove the classification head by replacing it with an identity layer
model.classifier = nn.Identity()

# Set up device and multi-GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = nn.DataParallel(model, device_ids=[0, 1, 2, 3])
model.to(device)

# Set model to eval mode
model.eval()

DataParallel(
  (module): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        )
        (1): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNorm2d(

In [6]:
def extract_features_from_video(video_path, model, preprocess, device, num_frames=60):
    """
    Extract feature vectors for a video by processing a fixed number of frames
    through EfficientNetB7.
    
    Args:
        video_path (str): Path to the video file.
        model (torch.nn.Module): EfficientNetB7 model (with classifier replaced).
        preprocess (torchvision.transforms.Compose): Preprocessing transforms.
        device (torch.device): Device to run the model on.
        num_frames (int): Number of frames to sample.
        
    Returns:
        numpy.ndarray: Feature vectors for each frame of shape (num_frames, feature_dim)
    """
    # Extract frames from the video
    frames = extract_frames_from_video(video_path, num_frames=num_frames)
    if not frames or len(frames) == 0:
        print(f"[Warning] No frames extracted from {video_path}")
        return None
    
    # Convert frames (numpy arrays) to PIL images and then apply preprocessing
    pil_frames = [Image.fromarray(frame) for frame in frames]
    frame_tensors = [preprocess(img) for img in pil_frames]

    if len(frame_tensors) == 0:
        print(f"[Warning] No preprocessed frames from {video_path}")
        return None
    
    # Stack the frame tensors to create a batch of shape: (num_frames, 3, 600, 600)
    batch_tensor = torch.stack(frame_tensors).to(device)
    
    # Pass the entire batch through the model to extract features
    with torch.no_grad():
        features = model(batch_tensor)
    
    return features.cpu().numpy()

In [ ]:
video_paths = "dataset/daisee-separated/Train/"

labels = []
video_features = []

for lbl in os.listdir(video_paths):
    class_path = os.path.join(video_paths, lbl)
    label = int(lbl)
    print("extracting video features for class", label)
    
    for video in tqdm(os.listdir(class_path)):
        if video.endswith(".mp4") or video.endswith(".avi"):
            video_path = os.path.join(class_path, video)
            video_features.append(extract_features_from_video(video_path, model, preprocess, device))
            labels.append(label)

    if label == 0 or label == 1:
        print("extracting augmented video features for class", label)
        aug_path = os.path.join(class_path, "Augmented")
        for video in tqdm(os.listdir(aug_path)):
            if video.endswith(".mp4") or video.endswith(".avi"):
                video_path = os.path.join(aug_path, video)
                video_features.append(extract_features_from_video(video_path, model, preprocess, device))
                labels.append(label)


# To save the list:
with open('efficientNet_features/daisee_train_video_features_face_v2_aug_60_2.pkl', 'wb') as f:
    pickle.dump(video_features, f)       

extracting video features for class 0


100%|██████████| 34/34 [02:00<00:00,  3.55s/it]


extracting video features for class 1


100%|██████████| 214/214 [12:41<00:00,  3.56s/it]


extracting video features for class 3


 14%|█▍        | 366/2585 [22:04<2:33:47,  4.16s/it][mpeg4 @ 0xd970e00] P cbpy damaged at 19 22
[mpeg4 @ 0xd970e00] Error at MB: 921
[mpeg4 @ 0x1c506480] P cbpy damaged at 19 22
[mpeg4 @ 0x1c506480] Error at MB: 921
[mpeg4 @ 0x1c506480] P cbpy damaged at 19 22
[mpeg4 @ 0x1c506480] Error at MB: 921
[mpeg4 @ 0xd908940] P cbpy damaged at 19 22
[mpeg4 @ 0xd908940] Error at MB: 921
[mpeg4 @ 0xd908940] P cbpy damaged at 19 22
[mpeg4 @ 0xd908940] Error at MB: 921
[mpeg4 @ 0x1c58cbc0] P cbpy damaged at 19 22
[mpeg4 @ 0x1c58cbc0] Error at MB: 921
[mpeg4 @ 0x1c58cbc0] P cbpy damaged at 19 22
[mpeg4 @ 0x1c58cbc0] Error at MB: 921
[mpeg4 @ 0x1c58cbc0] P cbpy damaged at 19 22
[mpeg4 @ 0x1c58cbc0] Error at MB: 921
 58%|█████▊    | 1511/2585 [1:32:01<1:08:07,  3.81s/it][mpeg4 @ 0x219d1780] I cbpy damaged at 16 8
[mpeg4 @ 0x219d1780] Error at MB: 344
[mpeg4 @ 0x23dfc040] I cbpy damaged at 16 8
[mpeg4 @ 0x23dfc040] Error at MB: 344
[mpeg4 @ 0x23dfc040] I cbpy damaged at 16 8
[mpeg4 @ 0x23dfc040] Error 

extracting video features for class 2


100%|██████████| 2649/2649 [2:36:26<00:00,  3.54s/it]  


In [8]:
video_paths = "dataset/daisee-separated/Validation/"

labels = []
video_features = []

for lbl in os.listdir(video_paths):
    class_path = os.path.join(video_paths, lbl)
    label = int(lbl)
    print("extracting video features for class", label)
    
    for video in tqdm(os.listdir(class_path)):
        if video.endswith(".mp4") or video.endswith(".avi"):
            video_path = os.path.join(class_path, video)
            video_features.append(extract_features_from_video(video_path, model, preprocess, device))
            labels.append(label)
        
            
# To save the list:
with open('efficientNet_features/daisee_val_video_features_face_v2_aug_60.pkl', 'wb') as f:
    pickle.dump(video_features, f)   

extracting video features for class 0


100%|██████████| 23/23 [01:14<00:00,  3.24s/it]


extracting video features for class 1


100%|██████████| 160/160 [09:37<00:00,  3.61s/it]


extracting video features for class 3


100%|██████████| 625/625 [38:28<00:00,  3.69s/it]


extracting video features for class 2


100%|██████████| 912/912 [51:46<00:00,  3.41s/it]  


In [9]:
video_paths = "dataset/daisee-separated/Test/"

labels = []
video_features = []

for lbl in os.listdir(video_paths):
    class_path = os.path.join(video_paths, lbl)
    label = int(lbl)
    print("extracting video features for class", label)
    
    for video in tqdm(os.listdir(class_path)):
        if video.endswith(".mp4") or video.endswith(".avi"):
            video_path = os.path.join(class_path, video)
            video_features.append(extract_features_from_video(video_path, model, preprocess, device))
            labels.append(label)
        
            
# To save the list:
with open('efficientNet_features/daisee_test_video_features_face_v2_aug_60.pkl', 'wb') as f:
    pickle.dump(video_features, f)    


extracting video features for class 0


100%|██████████| 4/4 [00:12<00:00,  3.18s/it]


extracting video features for class 1


100%|██████████| 84/84 [04:36<00:00,  3.29s/it]


extracting video features for class 3


100%|██████████| 814/814 [51:00<00:00,  3.76s/it]  


extracting video features for class 2


100%|██████████| 882/882 [47:59<00:00,  3.26s/it]  
